In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

<font style = 'font-family:calibri'><font size = 6><b> U.S.A. Minimum Wage and Median Home Price<b></font>
### <i> Have U.S. wages kept up with the cost of home prices?

<font style = 'font-family:calibri'><font size = 4>In the news, a frequent line of the "pricing out of the middle class" can be repeatedly heard and read across many headlines. Places like San Jose, San Francisco, New York City, Los Angeles, and Honolulu always make the news for the skyrocketing prices of homes and how unaffordable these cities are becoming to the average worker. Is this also true for the United States as a whole; are home prices increasing at a faster rate then the median wage?
<p>For this analysis of wage and median home prices the datasets chosen are from Kaggle's <a href='https://www.kaggle.com/lislejoem/us-minimum-wage-by-state-from-1968-to-2017'><i>US Minimum Wage by State from 1968 to 2017 </i> </a> and the United States Census Bureau's Historical Data on monthly <a href='https://www.census.gov/construction/nrs/historical_data/index.html'><i>Median and Average Sale Price of New Houses Sold</i></a>. First, we will explore the minimum wage in the United States, then median home price values before determining if housing prices are out pacing wages.

In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
import ipywidgets as widgets
from scipy import special
from pandas import Series
import plotly as py
import plotly.graph_objs as go
import math

py.offline.init_notebook_mode(connected=True)

## <br><font style = 'font-family:calibri'><font size = 6>Minimum Wages by State

<font style = 'font-family:calibri'><font size = 4>The data set Minimum Wage includes the minimum wage of some U.S. territories and Washington D.C. from the years 1968 to 2017.</font>
<p>
    <font size = 3><ul><u><b>Chart Values</u> </b>(<a href='https://www.kaggle.com/lislejoem/us-minimum-wage-by-state-from-1968-to-2017'>from Kaggle</a>):
<p class='tab'><b>Year</b>: The year of the data.
<p><b>State</b>: The state or territory of the data.
<p><b>Table_Data</b>: The scraped value from the source.
<p><b>Footnote</b>: The footnote associated with the Table_Data. See more below in the dataset description. You can alternatively find them at the bottom of the US Department of Labor's page.
<p><b>High_Value</b>: As there were some values in Table_Data that had multiple values (usually associated with footnotes), this is the higher of the two values in the table. It could be useful for viewing the proposed minimum wage, because in most cases, the higher value meant that all persons protected under minimum wage laws eventually had minimum wage set at that value.
<p><b>Low_Value</b>: This is the same as High.Value, but has the lower of the two values. This could be useful for viewing the effective minimum wage at the year of setting the minimum wage, as peoples protected under such minimum wage laws made that value during that year (although, in most cases, they had a higher minimum wage after that year).
<p><b>CPI_Average</b>: This is the Consumer Price Index associated with that year. It was used to calculate 2018 equivalent values.
<p><b>High_2018</b>: This is the 2018 equivalent dollars for High_Value.
<p><b>Low_2018</b>: This is the 2018 equivalent dollars for Low_Value.

In [3]:
# Importing file with wage data
wage = pd.read_csv('Minimum Wage Data.csv', encoding = "Windows-1252" )

# Converting the State column from series to string data
wage['State'].to_string


# Printing the data frame
wage.head()

,Year,State,Table_Data,Footnote,High_Value,Low_Value,CPI_Average,High_2018,Low_2018
0,1968,Alabama,...,NaN,0.00000,0.00000,34.783333,0.00,0.00
1,1968,Alaska,2.1,NaN,2.10000,2.10000,34.783333,15.12,15.12
2,1968,Arizona,18.72 - 26.40/wk(b),(b),0.66000,0.46800,34.783333,4.75,3.37
3,1968,Arkansas,1.25/day(b),(b),0.15625,0.15625,34.783333,1.12,1.12
4,1968,California,1.65(b),(b),1.65000,1.65000,34.783333,11.88,11.88


<br><font style = 'font-family:calibri'><font size = 4>For this analysis, the territories will be removed as the <i>Median Home Wage</i> data set does not take them into account when calculating the monthly median home price.

In [4]:
# Removing all rows that do not contain the 50 U.S. States but leaving the Federal minimum wage (FLSA).
wage = wage[~wage['State'].isin(['Guam','District of Columbia','Puerto Rico','U.S. Virgin Islands'])]
wage.head()

,Year,State,Table_Data,Footnote,High_Value,Low_Value,CPI_Average,High_2018,Low_2018
0,1968,Alabama,...,NaN,0.00000,0.00000,34.783333,0.00,0.00
1,1968,Alaska,2.1,NaN,2.10000,2.10000,34.783333,15.12,15.12
2,1968,Arizona,18.72 - 26.40/wk(b),(b),0.66000,0.46800,34.783333,4.75,3.37
3,1968,Arkansas,1.25/day(b),(b),0.15625,0.15625,34.783333,1.12,1.12
4,1968,California,1.65(b),(b),1.65000,1.65000,34.783333,11.88,11.88


<br><font style = 'font-family:calibri'><font size = 4>I am also dropping the columns <i>Table_Data</i> and <i>Footnote</i> as they are not necessary for this analysis.

In [5]:
# Dropping Footnote and Table_Data column from data set.
wage = wage.drop(columns = ['Footnote','Table_Data'])
wage.head()

,Year,State,High_Value,Low_Value,CPI_Average,High_2018,Low_2018
0,1968,Alabama,0.00000,0.00000,34.783333,0.00,0.00
1,1968,Alaska,2.10000,2.10000,34.783333,15.12,15.12
2,1968,Arizona,0.66000,0.46800,34.783333,4.75,3.37
3,1968,Arkansas,0.15625,0.15625,34.783333,1.12,1.12
4,1968,California,1.65000,1.65000,34.783333,11.88,11.88


<br><font style = 'font-family:calibri'><font size = 4> Below we see the a scatter plot of the <i>High_Value</i> and <i>Year</i>. Each state is plotted once each year. From the plot we can see that there is an upward trend and that the minimum wage has steadily increased. There are some states like Tennessee, South Carolina, and Mississippi (among others) that do not have any minimum wage reported or they do not have state laws to dictate minimum wage. For these states, the set Federal Minimum Wage applies.
<br>
    <br>
We can see upon hovering that the Disctrict of Columbia is somewhat of an outlier as it has a higher minimum wage than most states in several years (1988,1992 and 2017). The recent increases in the district are because it is tied to the Consumer Price Index and <a href='https://www.minimum-wage.org/district-of-columbia'>as prices increase</a>, so does the minimum wage.

In [6]:
layout = go.Layout(
    title= 'Wage Over Time: by State and Year',
    yaxis = dict(
        title = 'Wage in U.S. $',
    ),
    xaxis = dict(
        title = 'Year'
    ),
    annotations=[
        dict(
            x=2017,
            y=11,
            xref='x',
            yref='y',
            text='Washington',
            showarrow=True,
            bgcolor= 'yellow',
            arrowhead=3,
            ax=0,
            ay=-40,
        ),
    ]
)

trace1 = go.Scatter(
    x = wage['Year'],
    y = wage['High_Value'],
    mode = 'markers',
    text = wage['State'],
    marker = dict(
        size = 10,
        color = np.random.randn(5000),
        colorscale = 'Viridis'),
    name = 'Wage Over Time',
    line = dict(
        shape = 'spline'
    )
)


fig = go.Figure (data = [trace1], layout = layout)
py.offline.iplot(fig)

## <br><br><br><font style = 'font-family:calibri'><font size = 6>Minimum Wages by Year

<font style = 'font-family:calibri'><font size = 4>To better analyze the minimum wage, lets consolidate all the states and the Federal Minimum wage to one mean value for each year.
    <p> The chart preview below calculates the mean instance for the values <i>High_Value, Low_Value, CPI_Average, High_2018</i>, and <i>Low_ 2018</i> for all states and D.C. then groups them by year.
    <p> Note that the value for <i>CPI_Average</i> remains the same because the value displayed in the original data set was already the average calculated value for that given year, repeated every year for each state. 

In [7]:
# Computing the yearly mean wage each year in the United States.
mean_yearly_wage = wage.groupby(['Year'], as_index=False).mean()
mean_yearly_wage.head()

,Year,High_Value,Low_Value,CPI_Average,High_2018,Low_2018
0,1968,0.889534,0.834299,34.783333,6.404510,6.006863
1,1969,0.889534,0.834299,36.683333,6.072157,5.695490
2,1970,0.972745,0.917020,38.825000,6.273137,5.913333
3,1971,0.972745,0.917020,40.491667,6.014314,5.669804
4,1972,1.171765,1.109176,41.816667,7.016471,6.641569


<br><font style = 'font-family:calibri'><font size = 4>The graph below displays the mean wage over time, rounded to the closest penny. We can see that wage has increased over time from .89 cents in 1968 to $7.50 in 2017, which is .25 cents higher than the <a href='https://www.dol.gov/whd/minimumwage.htm'> current Federal Minimum Wage</a>. It is likely that this discrepency is due to the high minimum wage in states like Washington, even though some states do not report their own minimum wage.

The year 1991 sees a jump in mean wage from the steady increase over time. This is likely due to the high increase of the minimum wage in Washington, Oregon, and Alaska that happened that same year (see graph <i>Wage Over Time: by State and Year</i>).

<br>
Though the Federal Minimum Wage has not increased since 2009, we can see that the mean wage in the U.S. has continued to increase. This is likely in relation to states adapting their own minimum wage that is higher than the federal mandated one. In the graph <i>Wage Over Time: by State and Year</i>, we can see that most of the states are clusterd closer together in the earlier years between 1968-1975 and 1980-1990 and as wages begin to vary and increase, so does the mean wage in the U.S.

In [8]:
layout = go.Layout(
    title= 'Wage Over Time: By Year',
    yaxis = dict(
        title = 'Wage in U.S. $',
    ),
    xaxis = dict(
        title = 'Year'
    )
)
tracemedian = go.Scatter(
    x = mean_yearly_wage['Year'],
    y = round(mean_yearly_wage['High_Value'],2),
    mode = 'markers',
    text = mean_yearly_wage['Year'],
    marker = dict(
        size = 10,
        color = '#66B7A3',
        ),
    name = 'Wage Over Time',
    line = dict(
        shape = 'spline'
    )
)

fig = go.Figure (data = [tracemedian], layout = layout)
py.offline.iplot(fig)

<br><font style = 'font-family:calibri'><font size = 4>Next, we can compare the mean wage in each given year to the 2018 equivalent wage, <i>High_2018</i>. The value of any given wage if it was earned in 2018. So the wage of .89 cents is is the 2018 equivalant of $6.40.

<br>Note that over time the mean minimum wage and the 2018 equivalent get closer together as wages no longer need to be adjusted for inflation. As we have seen on the above graphs, the minimum wage increases at a steady pace with some short periods of no growth. Looking at the <i>2018 Wage Equivalent</i> overlayed, we can see that there are some points on the line where the 2018 wage equivalent drops down closer to the real wage.

<br>Generally, the declines in equivalent value seems to coincide with a <a href='https://www.thebalance.com/the-history-of-recessions-in-the-united-states-3306011'> recession</a>, with the exception of 2008's <a href='https://www.thebalance.com/the-great-recession-of-2008-explanation-with-dates-4056832'>Great Recession</a>. The reason for the drop is probably because the value of the U.S. dollar at those points in time had decreased in value and the purchasing power dropped, even though minimum wages continued to rise. In terms of 2018 dollars, that money did not go as far.

In [9]:
layout = dict(title = 'Mean Wage Over Time: A Comparison To 2018 Value',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Wage in U.S. $'),
              annotations=[
                dict(
                    x=1973,
                    y=6.61,
                    xref='x',
                    yref='y',
                    text='OPEC Oil Embargo',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=4,
                    ax=0,
                    ay=70
                ),
                dict(
                    x=1980,
                    y=6.37,
                    xref='x',
                    yref='y',
                    text='Interest Rate Hike',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=4,
                    ax=0,
                    ay=-50
                ),             
                dict(
                    x=1990,
                    y=4.96,
                    xref='x',
                    yref='y',
                    text='Savings/Loan Crisis',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=4,
                    ax=0,
                    ay=30
                ),        
                dict(
                    x=2008,
                    y=6.88,
                    xref='x',
                    yref='y',
                    text='Great Recession',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=4,
                    ax=0,
                    ay=-45,
                ),
                dict(
                    x=1970,
                    y=6.27,
                    xref='x',
                    yref='y',
                    text='1970 Recession',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=4,
                    ax=0,
                    ay=-45,
                ),                                       
                dict(
                    x=2001,
                    y=6.25,
                    xref='x',
                    yref='y',
                    text='Dot-Com Crash',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=4,
                    ax=0,
                    ay=-45,
                ),                         
              ]            
        )


# Create a trace
trace5 = go.Scatter(
     x = mean_yearly_wage['Year'],
     y = round(mean_yearly_wage['High_Value'],2),
     name = 'High Yearly Wage',
     line = dict(
         color = ('rgb(102, 183, 163)')
     )
)     

trace6 = go.Scatter(
     x = mean_yearly_wage['Year'],
     y = round(mean_yearly_wage['High_2018'],2),
     name = '2018 Wage Equivelant',
     line = dict(
         color = ('rgb(184, 232, 16)')
     )
)

data = [trace5,trace6]

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')

<br><font style = 'font-family:calibri'><font size = 4>We see that the median wage in the United States has continued to increase since 1968. But is the income still enough to purchase a home?

## <br><br><br>Median Home Prices

<font style = 'font-family:calibri'><font size = 4> The <a href='https://www.census.gov/construction/nrs/historical_data/index.html'>United States Census Bureau</a> keeps records of home sales across the country. The below data set is for  new, single-family homes only from 1963 to 2011. The Survey of Construction does not collect sales information for multifamily buildings or for existing homes.

<p><font size = 3><ul><u><b>Chart Values:</u></b>
<p><b>Month</b>: Month of year.
<p><b>Year</b>: Year.
<p><b>Median_Home_Price</b>: The median home price of of single-family homes in each given month.
<p><b>Average_Home_Price</b>: The footnote associated with the Table_Data. See more below in the dataset description. You can alternatively find them at the bottom of the US Department of Labor's page.
<p><b>Adjusted Home</b>: this is the medican home value in thousands. This value was added in to the set to better compare against CPI.
       

In [10]:
# Importing file with median price of new homes.
median_home = pd.read_csv('median_home_price.csv')
median_home['Adjusted Home'] = median_home['Median_Home_Price']/1000
median_home.head(12)

,Month,Year,Median_Home_Price,Average_Home_Price,Adjusted Home
0,Jan,1963,17200,(NA),17.2
1,Feb,1963,17700,(NA),17.7
2,Mar,1963,18200,(NA),18.2
3,Apr,1963,18200,(NA),18.2
4,May,1963,17500,(NA),17.5
5,Jun,1963,18000,(NA),18.0
6,Jul,1963,18400,(NA),18.4
7,Aug,1963,17800,(NA),17.8
8,Sep,1963,17900,(NA),17.9
9,Oct,1963,17600,(NA),17.6


<br><font style = 'font-family:calibri'><font size = 4>The below graph displays the median home prices from 1963 to 2011. Like median wages, home prices have continued to grow over time. Between 1990 and 2007, we can see that home prices rose faster than the between 1963 to 1980 as the slope of the graph becomes steeper at this time. In 2007, there is a sharp drop in home prices that can be attributed to the <a href='https://www.thebalance.com/subprime-mortgage-crisis-effect-and-timeline-3305745'>subprime mortgage crisis</a>, a part of the Great Recession. 
<p>Overlayed is the average Consumer Price Index (CPI) from the Median Wage data set above. The CPI has a steadier slope than median home prices. Unlike wages and home prices, we do not see any real dips. This most likely because of the way <a href='https://www.thebalance.com/consumer-price-index-cpi-index-definition-and-calculation-3305735'>CPI is calculated</a> and items are replaced according to consumer trends. Between the years 2003 and 2008, we can see that home prices outpaced CPI. Some may attribute this rise to the lack of available homes at that time as well as the <a href='https://positivemoney.org/issues/house-prices/'>increased lending from banks</a>.
<p>

In [11]:
layout = go.Layout(
    title= 'Median Home Value Over Time',
    yaxis = dict(
        title = 'Home Value in U.S. $',
    ),
    xaxis = dict(
        title = 'Year'
    ),
    annotations=[
                dict(
                    x=2008,
                    y=260,
                    xref='x',
                    yref='y',
                    text='2008 Great Recession',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=1,
                    ax=10,
                    ay=-70
                ),
                
            ]
    )

trace1 = go.Scatter(
    x = median_home['Year'],
    y = median_home ['Adjusted Home'],
    mode = 'markers',
    marker = dict(
        size = 10,
        color = np.random.randn(5000),
        colorscale = 'Viridis'),
    name = 'Wage Over Time',
    line = dict(
        shape = 'spline'
    )
)
trace0 = go.Scatter(
     x = mean_yearly_wage['Year'],
     y = round(mean_yearly_wage['CPI_Average'],2),
     name = 'Average CPI',
     line = dict(
         color = 'red',
     )
)

fig = go.Figure (data = [trace1, trace0], layout = layout)
py.offline.iplot(fig)

<br><font style = 'font-family:calibri'><font size = 4>By taking the yearly median home value we can better analyze the the trends in home prices. 

In [12]:
# Computing the yearly mean wage each year in the United States.
mean_yearly_home = median_home.groupby(['Year'], as_index = False).mean()
mean_yearly_home.head()

,Year,Median_Home_Price,Adjusted Home
0,1963,17966.666667,17.966667
1,1964,18983.333333,18.983333
2,1965,20150.000000,20.150000
3,1966,21525.000000,21.525000
4,1967,22691.666667,22.691667


<br><font style = 'font-family:calibri'><font size = 4>In the below graph CPI and home prices are again displayed, however, the slope of home prices is more discernable. In this graph we can see that home prices actually stabalized in 2006, two years before the housing bubble burst. We can see that 2009 was the lowest point in value for new, single-family homes.
<p>Again we see that home values are significantly higher than CPI from 2003 to 2008. Though CPI was increasing and seemed healthy, it was not able to predict the housing market crash. This is likely because of the way housing is calculated, where it calculates the monthly cost of owning a home based on rent prices. As we saw during the Great Recession, the number of vacant homes increased, causing the rental prices to also increase as new renters flooded the market. This means that the CPI was given a false low reading during the housing boom, pre-Great Recession.

In [13]:
layout = dict(title = 'Yearly Median Home Value (<i>New, Single-Family Homes</i>) and CPI Over Time',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = '$ U.S. '),
            annotations=[
                dict(
                    x=2006,
                    y=244,
                    xref='x',
                    yref='y',
                    text='Home prices stabalize',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=1,
                    ax=10,
                    ay=-70
                ),
                dict(
                    x=2009,
                    y=213,
                    xref='x',
                    yref='y',
                    text='Lowest home values',
                    showarrow=True,
                    bgcolor= 'yellow',
                    arrowhead=1,
                    ax=10,
                    ay=70
                ),
            ]
    )


# Create a trace
trace7 = go.Scatter(
     x = mean_yearly_wage['Year'],
     y = mean_yearly_wage['CPI_Average'],
     name = 'Average CPI',
     line = dict(
         color = ('rgb(102, 183, 163)')
     )
)     

trace8 = go.Scatter(
     x = mean_yearly_home['Year'],
     y = round(mean_yearly_home['Adjusted Home'],2),
     name = 'Home Price Displayed in $1000s',
     line = dict(
         color = 'red'
     )
)

data = [trace7,trace8]

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')


## <br><br><br> Wage and Median Home Prices


<font style = 'font-family:calibri'><font size = 4> By combining the created data sets <i>Median Wage</i> and <i>Median Home Prices</i> we can create two new columns with the percent change of values from <i>High_Value</i>(median minimum wage in a given year) and <i>Median_Home_Price</i>(new, single-family home prices). See columns in data set below <b>PercentHome_Change</b> and <b>PercentWage_Change</b>.
<p>
<font size = 3><ul><u><b>Added Chart Values</u>:</b>
<p class='tab'><p><b>PercentHome_Change</b>: Percent change from previous year for median minimum wage.
<p><b>PercentWage_Change</b>: Percent change from previous year for median prices of new, single-family homes. 


In [14]:
# Merging both data sets wage and median home.
wage_and_medianhome = mean_yearly_wage.merge(mean_yearly_home, on = 'Year')


wage_and_medianhome['PercentHome_Change'] = wage_and_medianhome['Median_Home_Price'].pct_change(1)
wage_and_medianhome['PercentWage_Change'] = wage_and_medianhome['High_Value'].pct_change(1)

print(wage_and_medianhome.shape)
wage_and_medianhome.head()

(44, 10)


,Year,High_Value,Low_Value,CPI_Average,High_2018,Low_2018,Median_Home_Price,Adjusted Home,PercentHome_Change,PercentWage_Change
0,1968,0.889534,0.834299,34.783333,6.404510,6.006863,24833.333333,24.833333,NaN,NaN
1,1969,0.889534,0.834299,36.683333,6.072157,5.695490,25575.000000,25.575000,0.029866,0.000000
2,1970,0.972745,0.917020,38.825000,6.273137,5.913333,23533.333333,23.533333,-0.079831,0.093544
3,1971,0.972745,0.917020,40.491667,6.014314,5.669804,25216.666667,25.216667,0.071530,0.000000
4,1972,1.171765,1.109176,41.816667,7.016471,6.641569,27550.000000,27.550000,0.092531,0.204596


<br><font style = 'font-family:calibri'><font size = 4>The graph below, <i>Minimum Wage and Median Single-Family Home Price: Yearly Percent Change</i>, is a better visual representation of the changes in each year. Here, we can compare the changes in wages and homes each year. As we can see, percent wage change varies over the years, but never dips below zero. This supports the continual upward trend seen in graph <i>Wage Over Time: By Year</i>. In contrast, percent change for new, single-family home values dip below zero on four seperate occasions. These occasions align with the 1970's Recession, savings/loan crisis, and the Great Recession. 
<p>
Notably, there seems to be greater increases in wages before 1995. After 1995, the changes are more moderate with the majority bieng less than a .1% change. At the same time, home values seem to be increasing at a higher percent than wages. Between the years 2001 and 2005, home values increased by at least twice or more percentage points than wages. Then the Great Recession happened and home prices plummetted before increasing again in 2010 and 2011.

In [15]:
layout = dict(title = 'Minimum Wage and Median Single-Family Home Prices: Yearly Percent Change',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Percent Change')
             )

    
data = [
    go.Bar(
        x = wage_and_medianhome['Year'],
        y = round(wage_and_medianhome['PercentHome_Change'],2),
        base = 0,
        marker = dict(
          color = ('rgb(184, 232, 16)')
        ),
        name = '% Home Price Change'
    ),
    go.Bar(
        x = wage_and_medianhome['Year'],
        y = round(wage_and_medianhome['PercentWage_Change'],2),
        base = 0,
        marker = dict(
          color = ('rgb(102, 183, 163)')
        ),
        name = '% Wage Change'
    )
]


fig = go.Figure(layout=layout,data=data)
py.offline.iplot(fig, filename='base-bar')

<br><font style = 'font-family:calibri'><font size = 4> Overall we can see that wages and new, single-family home prices have a similar trend of continual growth, despite some moments of decline. Further reseasearch is needed to truly assess if home values are increasing at a greater rate then wages. It would be beneficial to examine home prices by state rather than month. It might also be interesting to examine home prices for all types of homes, rather than just new, single-family homes as we did in this analysis. It would be intresting to note how each state varied in regards to its wages and home prices. Are some states better off than others in this regard? Do states that have higher minimum wages have higher home prices as well?